In [1]:
import dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from ragas import evaluate# 0.0.22
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
import os

c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
OPENAI_DEPLOYMENT = os.environ.get("OPENAI_DEPLOYMENT")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL")
EMBEDDING_DEPLOYMENT = os.environ.get("EMBEDDING_DEPLOYMENT")

# Evaluation part

### Ground truth

In [6]:
questions = [
    "Who was Alexei Navalny supposed to be exchanged for in the planned prisoner swap, according to his colleague Maria Pevchikh?",
    "What major issues with Boeing's safety culture were highlighted in the recent FAA report, and how is Boeing responding to the findings?",
    "What financial challenges is Europe facing in sustaining support for Ukraine, and how are officials suggesting addressing these challenges in the next 12 months?",
    "What is Sora AI, how does it work, and when can the general public expect to use it?",
    "What are the potential key issues that could impact the 2024 U.S. presidential election between Joe Biden and Donald Trump, and how might they impact the outcome?",
    "Why does Elon Musk want a bigger stake in Tesla, and what concerns does he express about the current structure of the company?",
    "What are the shopping habits of Gen Alpha, and why are mature brands interested in targeting this demographic?",
    "What is the main goal of Hamas in the Israel-Gaza conflict?",
    "What has Kim Jong Un recently declared about North Korea's relationship with South Korea, and what actions has he taken that raise concerns about potential military conflict?",
    "Why did Prince William pull out of the godfather's memorial service on 27.02.2024?"
]

ground_truths = [
    ["Alexei Navalny was supposed to be exchanged for Vadim Krasikov, a Russian hitman serving a life sentence for murder in Germany, as claimed by Maria Pevchikh, his colleague and chairwoman of his Anti-Corruption Foundation."],
    ["The FAA report on Boeing's safety culture found significant problems, including a 'disconnect' between senior management and employees, fear of retaliation when reporting safety concerns, confusion in safety reporting channels, and inadequate human factors in aviation safety. Boeing's response acknowledges the gaps, expressing commitment to learning from the findings, fostering a safety culture, and thoroughly reviewing the recommendations. The report identified 27 findings and 53 recommendations for improvement, which the FAA will review to ensure Boeing addresses them comprehensively."],
    ["Europe is grappling with the practicality of sustaining draining financial support for Ukraine amidst the deadlocked war and delayed aid. As spending on Ukraine faces political challenges, officials suggest exploring alternative funding sources, such as using frozen Russian assets to cover compensation costs. While some caution against setting a precedent, officials believe the EU's fundraising capabilities and potential arms deals, coupled with reduced reliance on the US, could help Europe continue supporting Ukraine in the coming year."],
    ["Sora AI is an artificial intelligence tool developed by OpenAI that can generate realistic videos up to 1 minute long based on textual prompts. It utilizes diffusion models, a method used in AI image generators, to create videos by understanding the association between images and accompanying alt text. As of now, Sora is not available to the general public; OpenAI is following a cautious approach by first testing it with a small group of 'red teamers' for potential harm or risks, followed by availability to visual artists, designers, and filmmakers before a potential public release, likely under a pay-to-use model similar to GPT. While Sora has shown significant advancements in AI video generation compared to previous attempts, it is not perfect, with occasional flaws and limitations seen in hand-picked videos released by OpenAI."],
    ["The potential key issues that could impact the 2024 U.S. presidential election between Joe Biden and Donald Trump include contrasting economic views, with Americans historically voting based on their economic well-being; the contentious topics of abortion and immigration, with Democrats focusing on abortion rights and Republicans emphasizing border security; and external factors like the Gaza War and its impact on Biden's support base. Other factors include crime rates, environmental concerns, and foreign policy. Additionally, the health and age of both candidates could be scrutinized, and the potential emergence of third-party candidates or independents might introduce unpredictability. Trump's legal challenges, facing 91 charges and four criminal trials, could influence public opinion, and the specter of the January 2021 Capitol attack may resonate differently among Republican and Democratic voters. Overall, these factors contribute to the complexity and uncertainty of the upcoming presidential race."],
    ["Elon Musk wants a bigger stake in Tesla to gain more control over the company's direction, especially regarding its investments in artificial intelligence (AI) features. He is concerned about Tesla's vulnerability to a 'takeover by dubious interests' and aims to have 25% voting control to ensure the company's leadership in AI and robotics. Musk suggests that without this control, he would prefer to develop products outside of Tesla."],
    ["Gen Alpha, born between 2010 and 2024, prefers to shop at adult brands like Lululemon, Sephora, Walmart, and Target, following the trends of their millennial parents. Adult brands are interested in targeting Gen Alpha due to their economic potential, with an estimated economic footprint of $5.46 trillion by 2029. These brands can secure the loyalty of the next generation by expanding their offerings to cater to the specific needs of Gen Alpha."],
    ["Hamas aims to create an Islamic state in place of Israel and rejects Israel's right to exist, justifying its attacks as responses to what it perceives as Israeli crimes against the Palestinian people. Additionally, Hamas seeks the release of Palestinian prisoners, an end to the blockade of the Gaza Strip, and the establishment of an independent Palestinian state."],
    ["Kim Jong Un recently declared that North Korea would no longer pursue reconciliation with South Korea, framing South Korea as the 'principal enemy.' He intensified nuclear threats, conducted missile tests, and abolished government agencies promoting cooperation and reunification. Analysts, including Robert L. Carlin and Siegfried Hecker, suggest he may be preparing for a military attack on South Korea."],
    ["Due to 'personal matter'."]
]
answers_llm = []
contexts_llm = [[""],[""],[""],[""],[""],[""],[""],[""],[""],[""]]

In [5]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION)
llm = AzureChatOpenAI(model_name=OPENAI_MODEL, azure_deployment=OPENAI_DEPLOYMENT,temperature=0)

In [7]:
def evaluation_llm(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
    )
    return result

In [8]:
def evaluation_rag(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  # most likely
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
    )
    return result

### General answers by LLM

In [11]:
template = """{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
llm_chain =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm}
)


In [13]:
for query in questions:
    response = llm_chain.invoke({"question": query})
    answers_llm.append(response["response"].content)

Who was Alexei Navalny supposed to be exchanged for in the planned prisoner swap, according to his colleague Maria Pevchikh?
What major issues with Boeing's safety culture were highlighted in the recent FAA report, and how is Boeing responding to the findings?
What financial challenges is Europe facing in sustaining support for Ukraine, and how are officials suggesting addressing these challenges in the next 12 months?
What is Sora AI, how does it work, and when can the general public expect to use it?
What are the potential key issues that could impact the 2024 U.S. presidential election between Joe Biden and Donald Trump, and how might they impact the outcome?
Why does Elon Musk want a bigger stake in Tesla, and what concerns does he express about the current structure of the company?
What are the shopping habits of Gen Alpha, and why are mature brands interested in targeting this demographic?
What is the main goal of Hamas in the Israel-Gaza conflict?
What has Kim Jong Un recently d

In [14]:
llm_results = evaluation_llm(questions, answers_llm, contexts_llm, ground_truths)
print(llm_results)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 40/40 [00:11<00:00,  3.60it/s]


{'faithfulness': 0.9583, 'answer_relevancy': 0.4750, 'answer_similarity': 0.9093, 'answer_correctness': 0.5995}


### Naive RAG

In [42]:
loader = DirectoryLoader('./news', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [43]:
text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db = Chroma.from_documents(chunks, embeddings_client)
retriever = db.as_retriever()

Created a chunk of size 4589, which is longer than the specified 4000
Created a chunk of size 4082, which is longer than the specified 4000


In [44]:
template = """User input {question}. 
context {context}"""
prompt = ChatPromptTemplate.from_template(template)

In [45]:
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [46]:
answers_naive = []
contexts_naive = []
for query in questions:
    response = retrieval_augmented_qa_chain.invoke({"question": query})
    # Access the response content
    answers_naive.append(response["response"].content)
    # Access the context content
    context_content = [context.page_content for context in response["context"]]
    contexts_naive.append(context_content)

In [47]:
result_naive_rag = evaluation_rag(questions, answers_naive, contexts_naive, ground_truths)
print(result_naive_rag)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:11<00:00,  5.38it/s]


{'faithfulness': 0.9815, 'answer_relevancy': 0.8237, 'context_precision': 0.9000, 'context_recall': 0.9200, 'answer_similarity': 0.9512, 'answer_correctness': 0.7298}


In [ ]:
# df = result.to_pandas()
# df

## try recursive text splitter

In [74]:
text_splitter = RecursiveCharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db = Chroma.from_documents(chunks, embeddings_client)
retriever = db.as_retriever()

In [75]:
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [76]:
answers_recursive = []
contexts_recursive = []
for query in questions:
    response = retrieval_augmented_qa_chain.invoke({"question": query})
    # Access the response content
    answers_recursive.append(response["response"].content)
    # Access the context content
    context_content = [context.page_content for context in response["context"]]
    contexts_recursive.append(context_content)

In [78]:
result_recursive = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
print(result_recursive)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  23%|██▎       | 14/60 [00:05<00:17,  2.62it/s]
Invalid JSON response. Expected dictionary with key 'question'
Exception in thread Thread-25:
Traceback (most recent call last):
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-code\.venv\Lib\site-packages\ragas\executor.py", line 75, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "c:\Users\

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

## chunk size change

In [56]:
def change_chunk_size(text_splitter):
    chunks = text_splitter.split_documents(documents)
    db = Chroma.from_documents(chunks, embeddings_client)
    retriever = db.as_retriever()

    retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
    )
    answers = []
    contexts = []

    for query in questions:
        response = retrieval_augmented_qa_chain.invoke({"question": query})
        # Access the response content
        answers.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts.append(context_content)

    result = evaluation_rag(questions, answers, contexts, ground_truths)
    return result

In [64]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 3500, chunk_overlap = 350)
result_3500 = change_chunk_size(text_splitter)
print("CHUNK SIZE 3500")
print(result_3500)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  28%|██▊       | 17/60 [00:05<00:13,  3.19it/s]
Invalid JSON response. Expected dictionary with key 'question'
Exception in thread Thread-22:
Traceback (most recent call last):
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-code\.venv\Lib\site-packages\ragas\executor.py", line 75, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "c:\Users\

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.